Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.  

In this assignment, we
- build a MLP classifier for the Fashion-MNIST dataset.
- use PCA to reduce the dimensionality of the dataset, and make sure we preserve 95% of the explained variance. (20 points)
- train a classifier using the dimensionality reduced dataset with the same network toplogy as the previous classfier,  and compare the classification accuracy result with the one using the original dataset. (10 points)
- check whether we observe anything surprising. (10 points)
- follow and improve the example from the text to fine tune the neural network hyperparameters using RandomizedSearchCV. Note to use the dataset after the PCA step. This will make the search less time consuming. (40 points)
- report the test result using the best model obtained from the randomized search. Show the summary of the model. Compare this result with the previous results. (20 points)

In [ ]:
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Read the data.

In [ ]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


Set random seeds.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(64, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
X_train_full = X_train_full.reshape((60000, 28 * 28))
X_train_full = X_train_full.astype('float32') / 255

X_test = X_test.reshape((10000, 28 * 28))
X_test = X_test.astype('float32') / 255

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_full = to_categorical(y_train_full)
y_test = to_categorical(y_test)

In [ ]:
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]

In [ ]:
network.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=128)

Epoch 1/20
430/430 [==============================] - 7s 13ms/step - loss: 0.6238 - accuracy: 0.7860 - val_loss: 0.4536 - val_accuracy: 0.8484
Epoch 2/20
430/430 [==============================] - 4s 9ms/step - loss: 0.4435 - accuracy: 0.8442 - val_loss: 0.3929 - val_accuracy: 0.8658
Epoch 3/20
430/430 [==============================] - 3s 6ms/step - loss: 0.4002 - accuracy: 0.8559 - val_loss: 0.4288 - val_accuracy: 0.8404
Epoch 4/20
430/430 [==============================] - 2s 5ms/step - loss: 0.3721 - accuracy: 0.8661 - val_loss: 0.3633 - val_accuracy: 0.8684
Epoch 5/20
430/430 [==============================] - 3s 7ms/step - loss: 0.3541 - accuracy: 0.8715 - val_loss: 0.3553 - val_accuracy: 0.8744
Epoch 6/20
430/430 [==============================] - 4s 8ms/step - loss: 0.3366 - accuracy: 0.8775 - val_loss: 0.3458 - val_accuracy: 0.8800
Epoch 7/20
430/430 [==============================] - 2s 5ms/step - loss: 0.3242 - accuracy: 0.8834 - val_loss: 0.3447 - val_accuracy: 0.8796
Epoch

In [ ]:
test_loss, test_acc = network.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3870 - accuracy: 0.8648


In [ ]:
print('test_acc:', test_acc)

test_acc: 0.864799976348877


Now we read the data again.

In [ ]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

Use the same random seeds.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
X_train_full.shape

(60000, 28, 28)

In [ ]:
X_test.shape

(10000, 28, 28)

In [ ]:
X_train_full = X_train_full.reshape((60000,28*28))
X_test = X_test.reshape((10000,28*28))

Conduct fit and transform on X_train_full using PCA. (10 points)

In [ ]:
from sklearn.decomposition import PCA

# fill in code here
pca = PCA(n_components=0.95)
X_train_reduced_full = pca.fit_transform(X_train_full)

Transform X_test using the PCA. (10 points)

In [ ]:
# fill in code here
X_test_reduced = pca.transform(X_test)

Fill in the input_shape in the following code. (10 points)

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
# fill in code
input_shape = (X_train_reduced_full.shape[1],)
network.add(layers.Dense(64, activation='relu', input_shape=input_shape))
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                12032     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 12682 (49.54 KB)
Trainable params: 12682 (49.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
X_train_reduced_full = X_train_reduced_full.astype('float32') / 255
X_test_reduced = X_test_reduced.astype('float32') / 255

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_full = to_categorical(y_train_full)
y_test = to_categorical(y_test)

In [ ]:
X_valid_reduced = X_train_reduced_full[:5000]
y_valid = y_train_full[:5000]
X_train_reduced = X_train_reduced_full[5000:]
y_train = y_train_full[5000:]

In [ ]:
X_train_reduced_full.shape

(60000, 187)

In [ ]:
y_train.shape

(55000, 10)

In [ ]:
network.fit(X_train_reduced, y_train, validation_data=(X_valid_reduced, y_valid), epochs=20, batch_size=128)

Epoch 1/20
430/430 [==============================] - 2s 4ms/step - loss: 0.6835 - accuracy: 0.7814 - val_loss: 0.4096 - val_accuracy: 0.8570
Epoch 2/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3903 - accuracy: 0.8613 - val_loss: 0.3660 - val_accuracy: 0.8704
Epoch 3/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3537 - accuracy: 0.8732 - val_loss: 0.3514 - val_accuracy: 0.8732
Epoch 4/20
430/430 [==============================] - 1s 2ms/step - loss: 0.3302 - accuracy: 0.8821 - val_loss: 0.3347 - val_accuracy: 0.8802
Epoch 5/20
430/430 [==============================] - 1s 2ms/step - loss: 0.3130 - accuracy: 0.8874 - val_loss: 0.3257 - val_accuracy: 0.8830
Epoch 6/20
430/430 [==============================] - 1s 2ms/step - loss: 0.2985 - accuracy: 0.8925 - val_loss: 0.3217 - val_accuracy: 0.8866
Epoch 7/20
430/430 [==============================] - 1s 2ms/step - loss: 0.2864 - accuracy: 0.8967 - val_loss: 0.3130 - val_accuracy: 0.8894
Epoch 

In [ ]:
test_loss, test_acc = network.evaluate(X_test_reduced, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3339 - accuracy: 0.8869


In [ ]:
print('test_acc:', test_acc)

test_acc: 0.886900007724762


Compare these two accuracy results and check whether we see anything surprising. (10 points)

The accuracy result ...

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

Modify the code provided by this module and use RandomizedSearchCV to find a model that beats the previous accuracy results. (40 points)

Hint: you can speed up the search by using n_jobs = 1 in RandomizedSearchCV.

In [ ]:
X_valid = X_train_reduced[:5000]
y_valid = y_train[:5000]
X_train = X_train_reduced[5000:]
y_train = y_train[5000:]

In [ ]:
X_train.shape, X_valid.shape

((50000, 187), (5000, 187))

In [ ]:
y_train.shape, y_valid.shape, y_test.shape

((50000, 10), (5000, 10), (10000, 10))

In [ ]:
from tensorflow import keras

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def build_model(n_hidden=1, n_neurons=128, learning_rate=3e-3, input_shape=(X_train_reduced.shape[1],)):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        # fill in code
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    # fill in code
    model.add(layers.Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer='rmsprop',
                  # fill in code
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [ ]:
from tensorflow import keras
from sklearn.base import BaseEstimator, RegressorMixin

# Modify the KerasRegressorWrapper class to accept hyperparameters
class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_hidden=1, n_neurons=200, learning_rate=1e-3):
        self.n_hidden = n_hidden
        self.n_neurons = n_neurons
        self.learning_rate = learning_rate

    def fit(self, X, y, **kwargs):
        self.model = build_model(self.n_hidden, self.n_neurons, self.learning_rate)
        self.model.fit(X, y, **kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Create an instance of the KerasRegressorWrapper
keras_reg = KerasRegressorWrapper()


In [ ]:
keras_reg.fit(X_train, y_train, epochs=100,
              # fill in code
              validation_data=( X_valid,y_valid ),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.4494 - accuracy: 0.8464 - val_loss: 0.3601 - val_accuracy: 0.8700
Epoch 2/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.3114 - accuracy: 0.8888 - val_loss: 0.3186 - val_accuracy: 0.8858
Epoch 3/100
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2689 - accuracy: 0.9035 - val_loss: 0.3105 - val_accuracy: 0.8904
Epoch 4/100
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2391 - accuracy: 0.9145 - val_loss: 0.3085 - val_accuracy: 0.8888
Epoch 5/100
1563/1563 [==============================] - 4s 3ms/step - loss: 0.2145 - accuracy: 0.9232 - val_loss: 0.3007 - val_accuracy: 0.8922
Epoch 6/100
1563/1563 [==============================] - 4s 3ms/step - loss: 0.1938 - accuracy: 0.9311 - val_loss: 0.3043 - val_accuracy: 0.8936
Epoch 7/100
1563/1563 [==============================] - 3s 2ms/step - loss: 0.1755 - accuracy: 0.9378 - val_loss: 0.3095 - val_ac

KerasRegressorWrapper()

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    # fill in code
    "n_hidden": [ 1, 2 ],
    "n_neurons": np.arange(200 , 500)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, verbose=2, n_jobs=-1)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  # fill in code
                  validation_data=( X_valid , y_valid ),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4292 - accuracy: 0.8517 - val_loss: 0.3514 - val_accuracy: 0.8736
Epoch 2/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.3016 - accuracy: 0.8924 - val_loss: 0.3123 - val_accuracy: 0.8872
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2572 - accuracy: 0.9079 - val_loss: 0.3061 - val_accuracy: 0.8902
Epoch 4/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.2256 - accuracy: 0.9203 - val_loss: 0.3036 - val_accuracy: 0.8918
Epoch 5/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.1987 - accuracy: 0.9291 - val_loss: 0.2980 - val_accuracy: 0.8964
Epoch 6/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.1755 - accuracy: 0.9380 - val_loss: 0.3048 - val_accuracy: 0.8960
Epoch 7/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.1561 - accuracy: 0.9461 - val_loss: 0.3153 - val_ac

RandomizedSearchCV(cv=3, estimator=KerasRegressorWrapper(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'n_hidden': [1, 2],
                                        'n_neurons': array([200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225,
       226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
       252, 253...
       408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420,
       421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433,
       434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459,
       460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472,
       473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485,
       486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498,
       499])},
                   verbose=2)

In [ ]:
rnd_search_cv.best_params_

{'n_neurons': 286, 'n_hidden': 1}

Show the summary of the best model obtained from the randomized search. Report the test result using the best model, and compare this result with the previous results. (20 points)

In [ ]:
# fill in code
model = rnd_search_cv.best_estimator_.model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 286)               53768     
                                                                 
 dense_3 (Dense)             (None, 10)                2870      
                                                                 
Total params: 56638 (221.24 KB)
Trainable params: 56638 (221.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.evaluate(X_test_reduced, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4731 - accuracy: 0.8880


[0.4731125831604004, 0.8880000114440918]

The result from the randomized search ...